## Importando bibliotecas

In [1]:
import re
import json
from nltk.metrics.distance import edit_distance
import numpy as np

## Lendo o dump de documentos

In [2]:
data = []
with open('dump_small.jsonln', 'r') as file:
    for line in file:
        data.append(json.loads(line))
        
print(f'Numero de documentos: {len(data)}')

Numero de documentos: 11225


In [3]:
print(data[0].keys())

dict_keys(['title', 'body'])


## Função para encontrar links HTML

In [4]:
# Baseado em https://gist.github.com/gruber/249502
def rm_htmlLinks(texto):
    return re.sub(r"(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))", "", texto)

## Função para limpar os wikilinks

In [5]:
def sub_links(texto):
    return re.sub(r"\[\[(?:[^|]*?\|)*?([^|]*?)\]\]", r"\1", texto)

## Função para limpar os templates

In [6]:
def rm_templates(texto):
    count = 0
    spansToRemove = []
    for item in re.finditer(r"{{|}}", texto):
        if item[0] == "{{":
            if count == 0:
                begin = item.span()[0]
            count += 1
        else:
            count -= 1
            if count == 0:
                end = item.span()[1]
                spansToRemove.append((begin, end))
    cleanText = ""
    begin = 0
    for span in spansToRemove:
        end, nextBegin = span
        cleanText += texto[begin:end]
        begin = nextBegin
    cleanText += texto[begin:]
    return cleanText

## Função para remover as ocorrências de ```{|...|}```

In [7]:
def rm_templates2(texto):
    count = 0
    spansToRemove = []
    for item in re.finditer(r"{[\|]|[\|]}", texto):
        if item[0] == "{|":
            if count == 0:
                begin = item.span()[0]
            count += 1
        else:
            count -= 1
            if count == 0:
                end = item.span()[1]
                spansToRemove.append((begin, end))
    cleanText = ""
    begin = 0
    for span in spansToRemove:
        end, nextBegin = span
        cleanText += texto[begin:end]
        begin = nextBegin
    cleanText += texto[begin:]
    return cleanText

## Função para remover as ocorrências de ```<ref>...</ref>```

In [8]:
def rm_refs(texto):
    return re.sub(r"<ref.*>.*</ref>|<ref.*>|</ref>", "", texto)

## Função pare remover as demais ocorrências de tags HTML

In [9]:
def rm_htmlTags(texto):
    return re.sub(r"<.*?>|</.*?>", "", texto)

## Função para remover as ocorrências de aspas (simples ou dupla)

In [10]:
def rm_aspas(texto):
    return re.sub(r"(['\"]+)(.*?)\1", r"\2", texto)

## Função para remover caracteres de pontuação ou semelhantes

In [11]:
def rm_caracterPontuacao(texto):
    sem_travessao = re.sub(r"\s\-\s|\-\-+", " ", texto)
    return re.sub(r"[^\w\s\-]", " ", sem_travessao)

## Função para remover as palavras que contém números

In [12]:
def rm_digitos(texto):
    return re.sub(r"\w*\d\w*", "", texto)

## Função para remover os travessões não hífen

In [13]:
def rm_travessaoErrado(texto):
    return re.sub(r"(\w+)\-|\-(\w+)", r"\1\2", texto)

## Função para remover palavras contendo caracteres fora do alfabeto português

In [14]:
def rm_notLatin(texto):
    return re.sub(r"[^a-zA-Z0-9\u00B5-\u00FF\s]", r"", texto)

## Pipeline Completa

In [15]:
def limpa_textos(txt):
    output = rm_htmlLinks(txt)
    output = sub_links(output)
    output = rm_refs(output)
    output = rm_htmlTags(output)
    output = rm_templates(output)
    output = rm_templates2(output)
    output = rm_aspas(output)
    output = rm_caracterPontuacao(output)
    output = rm_digitos(output)
    output = rm_travessaoErrado(output)
    output = rm_notLatin(output)
    return output.lower()

## Exemplo do resultado da limpeza

In [16]:
print(limpa_textos(data[0]["body"]))



alexandre é um prenome popular da língua portuguesa  é cognato ao nome alexander  da língua inglesa  em países lusófonos  pessoas chamadas alexandre são normalmente apelidadas de alex 

   origem   
o nome deriva do latim alexander  romanização do nome grego   aléksandros   etimologicamente  o nome seria um composto do verbo   aléksein   defender  e o substantivo   andrós   genitivo de   anr   homem  assim  pode ser traduzido como protetor da humanidade 

o termo seria ou um tipo raro de composto tatpurusha invertido  com o modificante na segunda posição  o tatpurusha cognato em sânscrito sendo  nararaka  cf  ramayana     o equivalente exato em sânscrito seria  rakinara  do pie hleks i hnros  ou um composto gasto do tipo terpsimbrotos  cujo significado original seria ele protege os homens 

o primeiro registro conhecido do nome foi feito no grego micênico  encontrouse a versão feminina do nome  alexandra  escrito em linear b 

o nome era um dos títulos  epítetos  dados à deusa grega 

## Calcula frequência absoluta de cada palavra

In [17]:
def define_palavras(docs):
    output = {}
    for doc in docs:
        words = limpa_textos(doc["body"]).split()
        for word in words:
            if word in output.keys():
                output[word] += 1
            else:
                output[word] = 1
    return output

In [18]:
%%time
palavras = define_palavras(data)
print(len(palavras))

159980
Wall time: 19 s


## Define o vocabulário com as 10 mil palavras mais frequentes

In [19]:
sort_palavras = dict(sorted(palavras.items(), key=lambda item: -item[1]))
vocab = list(sort_palavras.keys())[:10000]

## Corretor

In [22]:
LOWERCASE = [chr(x) for x in range(ord('a'), ord('z') + 1)]
LOWERCASE_OTHERS = ['ç', 'á', 'â', 'é', 'ã', 'õ', 'ê', 'í', 'ú', 'ô', 'ó']  # etc.
LETTERS = LOWERCASE + LOWERCASE_OTHERS

def edit1(text):
    words = []
    
    # Fase 1: as remoçoes.
    for p in range(len(text)):
        new_word = text[:p] + text[p + 1:]
        if len(new_word) > 0:
            words.append(new_word)
        
    # Fase 2: as adições.
    for p in range(len(text) + 1):
        for c in LETTERS:
            new_word = text[:p] + c + text[p:]
            words.append(new_word)
    
    # Fase 3: as substituições.
    for p in range(len(text)):
        orig_c = text[p]
        for c in LETTERS:
            if orig_c != c:
                new_word = text[:p] + c + text[p + 1:]
                words.append(new_word)
    
    return set(words)

def edit2(text):
    words1 = edit1(text)
    words2 = set()
    for w in words1:
        candidate_words2 = edit1(w)
        candidate_words2 -= words1
        words2.update(candidate_words2)
    words2 -= set([text])
    return words2

In [68]:
def candidatos(palavra):
    if palavra in vocab:
        candidatos = [palavra]
    else:
        candidatos = []
        dist1 = [w for w in edit1(palavra) if w in vocab]
        if len(dist1) > 0:
            candidatos += dist1
        else:
            dist2 = [w for w in edit2(palavra) if w in vocab]
            candidatos += dist2 + [palavra]
    return candidatos

In [69]:
def corrige(txt):
    frase = txt.split()
    output = ""
    for palavra in frase:
        candidates = candidatos(palavra.lower())
        melhorP = ""
        found = False
        for i in vocab:
            for e in candidates:
                if e == i:
                    melhorP = e
                    if palavra[0].isupper():
                        output += melhorP.capitalize() + " "
                        found = True
                        break
                    else:
                        output += melhorP + " "
                        found = True
                        break
            if found:
                break
        if not found:
            output += palavra + " "
    return output[:-1]

In [71]:
corrige("Três pratos de trigo para três tigres tristes")

'Três pratos de trigo para três tigre testes'

False